# NFL Stating out

In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

## Download the URL Page Content
The tables we are interested in are in javascript, so we need to use the webdriver from selenium to use chrome and get the page source. 

In [3]:
url = 'https://www.pro-football-reference.com/teams/mia/2020_roster.htm'
driver = webdriver.Chrome()
driver.get(url)
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')
driver.quit()


### Get a list of tables available on the URL 

In [4]:
Team = soup.find('h1').find_all()[1].get_text()
Tables = soup.find_all('table')
tableid = []
caption = []
for i in range(0,len(Tables)): 
    tableid.append(i)
    try: 
        caption.append(Tables[i].find_all('caption')[0].get_text())
    except Exception: 
        caption.append("No Caption")
Tables = pd.DataFrame({'tableid':tableid, 'tablename':caption})
Tables

,tableid,tablename
0,0,No Caption
1,1,No Caption
2,2,No Caption
3,3,No Caption
4,4,No Caption
5,5,No Caption
6,6,No Caption
7,7,No Caption
8,8,No Caption
9,9,No Caption


#### Passing Table

In [9]:
pd.read_html(page_source)[10]

,No.,Player,Age,Pos,G,GS,Wt,Ht,College/Univ,BirthDate,Yrs,AV,Drafted (tm/rnd/yr),Salary
0,34,De'Lance Turner,25,RB,NaN,NaN,214,5-11,Alcorn St.,8/23/1995,2,NaN,NaN,NaN
1,80,T.J. Rahming,23,WR,NaN,NaN,170,5-10,Duke,1/10/1997,Rook,NaN,NaN,NaN
2,1,Rashard Causey,24,DB,NaN,NaN,195,6-0,Central Florida,9/20/1996,Rook,NaN,NaN,NaN
3,45,Mike Hull,29,LB,NaN,NaN,232,6-0,Penn St.,5/25/1991,5,NaN,NaN,NaN
4,44,Deon Lacey,30,LB,NaN,NaN,235,6-3,West Alabama,7/18/1990,3,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,22,Tae Hayes,23,DB,NaN,NaN,188,5-9,Appalachian State,8/19/1997,1,NaN,NaN,NaN
81,89,Michael Roberts,26,TE,NaN,NaN,265,6-5,Toledo,5/7/1994,3,NaN,Detroit Lions / 4th / 127th pick / 2017,NaN
82,51,Kamu Grugier-Hill,26,OLB,NaN,NaN,230,6-2,East. Illinois,5/16/1994,4,NaN,New England Patriots / 6th / 208th pick / 2016,NaN
83,26,Steven Parker,25,DB,NaN,NaN,210,6-1,Oklahoma,12/14/1995,1,NaN,NaN,NaN


In [210]:
TeamStats = pd.read_html(page_source)[10]
columns = TeamStats.columns.to_frame()
filter = (columns.loc[:,0].str.contains("Rushing")) | (columns.loc[:,0].str.contains("Passing"))
columns.loc[filter,1] = columns[filter].loc[:,0] + "_" +columns[filter].loc[:,1]
NewColumns = columns.loc[:,1].to_list()
TeamStats.columns = NewColumns
TeamStats = TeamStats.rename(columns = {'Player':'Team'})
TeamStats.loc[TeamStats.Team == 'Team Stats','Team'] = Team
TeamStats = TeamStats[TeamStats['Team'] == Team]
TeamStats.head()



,Team,PF,Yds,Ply,Y/P,TO,FL,1stD,Passing_Cmp,Passing_Att,...,Yds,1stPy,#Dr,Sc%,TO%,Start,Time,Plays,Yds,Pts
0,Kansas City Chiefs,451,6067,976.0,6.2,15,10,350,378.0,576,...,1029.0,46.0,162.0,49.4,8.0,Own 28.9,2:52,6.18,37.2,2.62


In [211]:
Passing = pd.read_html(page_source)[13]
Rushing_Receiving = pd.read_html(page_source)[14]
columns = Rushing_Receiving.columns.to_frame()
filter = (columns.loc[:,0].str.contains("Rushing")) | (columns.loc[:,0].str.contains("Receiving"))
columns.loc[filter,1] = columns[filter].loc[:,0] + "_" +columns[filter].loc[:,1]
NewColumns = columns.loc[:,1].to_list()
Rushing_Receiving.columns = NewColumns
PlayerStats = Passing.merge(Rushing_Receiving, how = 'outer')
PlayerStats['Team'] = Team
PlayerStats.head()


,No.,Player,Age,Pos,G,GS,QBrec,Cmp,Att,Cmp%,...,Receiving_Lng,Receiving_R/G,Receiving_Y/G,Receiving_Ctch%,Receiving_Y/Tgt,Touch,Y/Tch,YScm,RRTD,Fmb
0,15.0,Patrick Mahomes*,24.0,QB,14,14.0,11-3-0,319.0,484.0,65.9,...,NaN,NaN,NaN,NaN,NaN,43.0,5.1,218.0,2.0,3.0
1,8.0,Matt Moore,35.0,NaN,6,2.0,1-1-0,59.0,91.0,64.8,...,NaN,NaN,NaN,NaN,NaN,5.0,-0.2,-1.0,0.0,2.0
2,2.0,Dustin Colquitt,37.0,NaN,16,0.0,NaN,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Team Total,26.5,NaN,16,NaN,12-4-0,378.0,576.0,65.6,...,83.0,23.6,293.1,67.7%,NaN,753.0,8.3,6259.0,46.0,20.0
4,NaN,Opp Total,NaN,NaN,16,NaN,NaN,352.0,582.0,60.5,...,NaN,22.0,221.4,NaN,NaN,NaN,NaN,5594.0,35.0,20.0


# Doing the loop


In [14]:
URL = ["https://www.pro-football-reference.com/teams/nwe/2019.htm"
,"https://www.pro-football-reference.com/teams/buf/2019.htm"
,"https://www.pro-football-reference.com/teams/nyj/2019.htm"
,"https://www.pro-football-reference.com/teams/mia/2019.htm"
,"https://www.pro-football-reference.com/teams/htx/2019.htm"
,"https://www.pro-football-reference.com/teams/oti/2019.htm"
,"https://www.pro-football-reference.com/teams/clt/2019.htm"
,"https://www.pro-football-reference.com/teams/jax/2019.htm"
,"https://www.pro-football-reference.com/teams/rav/2019.htm"
,"https://www.pro-football-reference.com/teams/pit/2019.htm"
,"https://www.pro-football-reference.com/teams/cle/2019.htm"
,"https://www.pro-football-reference.com/teams/cin/2019.htm"
,"https://www.pro-football-reference.com/teams/kan/2019.htm"
,"https://www.pro-football-reference.com/teams/den/2019.htm"
,"https://www.pro-football-reference.com/teams/rai/2019.htm"
,"https://www.pro-football-reference.com/teams/sdg/2019.htm"
,"https://www.pro-football-reference.com/teams/phi/2019.htm"
,"https://www.pro-football-reference.com/teams/dal/2019.htm"
,"https://www.pro-football-reference.com/teams/nyg/2019.htm"
,"https://www.pro-football-reference.com/teams/was/2019.htm"
,"https://www.pro-football-reference.com/teams/nor/2019.htm"
,"https://www.pro-football-reference.com/teams/atl/2019.htm"
,"https://www.pro-football-reference.com/teams/tam/2019.htm"
,"https://www.pro-football-reference.com/teams/car/2019.htm"
,"https://www.pro-football-reference.com/teams/gnb/2019.htm"
,"https://www.pro-football-reference.com/teams/min/2019.htm"
,"https://www.pro-football-reference.com/teams/chi/2019.htm"
,"https://www.pro-football-reference.com/teams/det/2019.htm"
,"https://www.pro-football-reference.com/teams/sfo/2019.htm"
,"https://www.pro-football-reference.com/teams/sea/2019.htm"
,"https://www.pro-football-reference.com/teams/ram/2019.htm"
,"https://www.pro-football-reference.com/teams/crd/2019.htm"]


URL = ["https://www.pro-football-reference.com/teams/nwe/2019.htm"
,"https://www.pro-football-reference.com/teams/buf/2019.htm"
,"https://www.pro-football-reference.com/teams/nyj/2019.htm"
,"https://www.pro-football-reference.com/teams/mia/2019.htm"]

In [15]:
TeamData = pd.DataFrame()
PlayerData = pd.DataFrame()
for i in URL: 
    url = i
    driver = webdriver.Chrome()
    driver.get(url)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    driver.quit()
    
    Team = soup.find('div', {'id':'info'}).find('h1').find_all('span')[1].get_text()
    TeamStats = pd.read_html(page_source)[10]
    columns = TeamStats.columns.to_frame()
    filter = (columns.loc[:,0].str.contains("Rushing")) | (columns.loc[:,0].str.contains("Passing"))
    columns.loc[filter,1] = columns[filter].loc[:,0] + "_" +columns[filter].loc[:,1]
    NewColumns = columns.loc[:,1].to_list()
    TeamStats.columns = NewColumns
    TeamStats = TeamStats.rename(columns = {'Player':'Team'})
    TeamStats.loc[TeamStats.Team == 'Team Stats','Team'] = Team
    TeamStats = TeamStats[TeamStats['Team'] == Team]
    TeamData = TeamData.append(TeamStats)
    
    Passing = pd.read_html(page_source)[13]
    Rushing_Receiving = pd.read_html(page_source)[14]
    columns = Rushing_Receiving.columns.to_frame()
    filter = (columns.loc[:,0].str.contains("Rushing")) | (columns.loc[:,0].str.contains("Receiving"))
    columns.loc[filter,1] = columns[filter].loc[:,0] + "_" +columns[filter].loc[:,1]
    NewColumns = columns.loc[:,1].to_list()
    Rushing_Receiving.columns = NewColumns
    PlayerStats = Passing.merge(Rushing_Receiving, how = 'outer')
    PlayerStats['Team'] = Team
    PlayerData = PlayerData.append(PlayerStats)
    
    import time
    time.sleep(30) 
    
TeamData.head()
PlayerData.head()

,No.,Player,Age,Pos,G,GS,QBrec,Cmp,Att,Cmp%,...,Receiving_R/G,Receiving_Y/G,Receiving_Ctch%,Receiving_Y/Tgt,Touch,Y/Tch,YScm,RRTD,Fmb,Team
0,12.0,Tom Brady,42.0,QB,16,16.0,12-4-0,373.0,613.0,60.8,...,NaN,NaN,NaN,NaN,26.0,1.3,34.0,3.0,4.0,New England Patriots
1,4.0,Jarrett Stidham,23.0,NaN,3,0.0,NaN,2.0,4.0,50.0,...,NaN,NaN,NaN,NaN,2.0,-1.0,-2.0,0.0,0.0,New England Patriots
2,11.0,Julian Edelman,33.0,WR,16,13.0,NaN,2.0,2.0,100.0,...,6.3,69.8,65.4%,7.3,108.0,10.6,1144.0,6.0,3.0,New England Patriots
3,28.0,James White,27.0,NaN,15,1.0,NaN,1.0,1.0,100.0,...,4.8,43.0,75.8%,6.8,139.0,6.5,908.0,6.0,1.0,New England Patriots
4,NaN,Team Total,28.0,NaN,16,NaN,12-4-0,378.0,620.0,61.0,...,23.6,259.6,64.4%,NaN,825.0,7.1,5856.0,42.0,13.0,New England Patriots


In [16]:
TeamData.head()

,Team,PF,Yds,Ply,Y/P,TO,FL,1stD,Passing_Cmp,Passing_Att,...,Yds,1stPy,#Dr,Sc%,TO%,Start,Time,Plays,Yds,Pts
0,New England Patriots,420,5664,1095.0,5.2,15,6,338,378.0,620,...,828.0,31.0,185.0,36.8,7.6,Own 32.5,2:40,5.97,29.9,1.99
0,Buffalo Bills,314,5283,1018.0,5.2,19,7,314,299.0,513,...,927.0,32.0,183.0,30.6,10.4,Own 28.9,2:34,5.62,28.6,1.66
0,New York Jets,276,4368,956.0,4.6,25,9,253,323.0,521,...,1105.0,30.0,183.0,23.0,11.5,Own 26.4,2:28,5.23,23.1,1.21
0,Miami Dolphins,306,4960,1022.0,4.9,26,8,315,371.0,615,...,769.0,41.0,180.0,30.6,13.3,Own 27.6,2:33,5.78,27.1,1.63
